In [35]:
import sys 
sys.path.append("C:/Users/isxzl/OneDrive/Code/AutoGeo")
import AutoGeo

In [39]:
from osgeo import gdal
# Replace this with your HDF5 file path
#hdf4_file_path = 'C:/Users/isxzl/OneDrive/Code/AutoGeo/data/MOD04_L2.A2023001.0210.061.2023001132858.hdf'
#hdf4_file_path = r'C:\Datasets\Zhejiang20-23RS\LandUse/MCD12Q1.A2019001.h28v05.061.2022169220530.hdf'
hdf4_file_path=r'C:/Users/isxzl/OneDrive/Code/AutoGeo/data/hdf4/MOD04_L2.A2023001.0350.061.2023001180312.hdf'


print("================================print product involved================================")


hdf4_dataset = gdal.Open(hdf4_file_path)
subdatasets = hdf4_dataset.GetSubDatasets()

for idx, subdataset in enumerate(subdatasets):
    print(f"{idx}: {subdataset[1]}")

print("================================metainfo for each product================================")

from prettytable import PrettyTable
import rasterio

# Create a PrettyTable object
table = PrettyTable()

# Initialize a counter for the index
index = 0

# Iterate over each subdataset
for subdataset in subdatasets:
    subdataset_path = subdataset[0]

    with rasterio.open(subdataset_path) as src:
        # Collect metadata and number of bands
        meta = src.meta
        meta['bands'] = src.count

        # Add field names to the table if it's the first iteration
        if not table.field_names:
            # Add 'index' at the beginning of the field names
            table.field_names = ['index'] + list(meta.keys())

        # Add the metadata to the table
        table.add_row([index] + list(meta.values()))

        # Increment the index
        index += 1

# Print the PrettyTable
print(table)


================================print product involved================================
0: [203x135] Scan_Start_Time mod04 (64-bit floating-point)
1: [203x135] Solar_Zenith mod04 (16-bit integer)
2: [203x135] Solar_Azimuth mod04 (16-bit integer)
3: [203x135] Sensor_Zenith mod04 (16-bit integer)
4: [203x135] Sensor_Azimuth mod04 (16-bit integer)
5: [203x135] Scattering_Angle mod04 (16-bit integer)
6: [203x135] Land_sea_Flag mod04 (16-bit integer)
7: [4060x2708] Aerosol_Cldmask_Land_Ocean mod04 (16-bit integer)
8: [4060x2708] Cloud_Pixel_Distance_Land_Ocean mod04 (16-bit integer)
9: [203x135] Land_Ocean_Quality_Flag mod04 (16-bit integer)
10: [203x135] Optical_Depth_Land_And_Ocean mod04 (16-bit integer)
11: [203x135] Image_Optical_Depth_Land_And_Ocean mod04 (16-bit integer)
12: [203x135] Average_Cloud_Pixel_Distance_Land_Ocean mod04 (16-bit integer)
13: [203x135] Aerosol_Type_Land mod04 (16-bit integer)
14: [203x135] Fitting_Error_Land mod04 (16-bit integer)
15: [3x203x135] Surface_Reflec

RasterioIOError: 'HDF4_EOS:EOS_SWATH:"C:/Users/isxzl/OneDrive/Code/AutoGeo/data/hdf4/MOD04_L2.A2023001.0350.061.2023001180312.hdf":mod04:Scan_Start_Time' does not exist in the file system, and is not recognized as a supported dataset name.

In [ ]:
from osgeo import gdal
import xarray as xr
import numpy as np

def hdf_to_xarray(hdf_file_path):
    # Open the HDF file
    hdf_dataset = gdal.Open(hdf_file_path)
    
    # List all subdatasets
    subdatasets = hdf_dataset.GetSubDatasets()
    
    # Initialize a dictionary to hold DataArrays for each subdataset
    data_arrays = {}
    
    # Loop through subdatasets
    for subdataset in subdatasets:
        # Get name and path
        path,name= subdataset
        
        # Open subdataset
        subdataset_ds = gdal.Open(path)
        
        # Read data into numpy array
        data = subdataset_ds.ReadAsArray()
        
        # Create an xarray DataArray, you can add coords and dims as needed
        # Here, we assume a simple 2D array without specific coordinates
        data_array = xr.DataArray(data, dims=['y', 'x'])
        
        # Add to our dictionary, using a cleaned up name
        clean_name = path.split(':')[-1]  # Simplify this as needed
        data_arrays[clean_name] = data_array
    
    # Combine all DataArrays into a single Dataset
    dataset = xr.Dataset(data_arrays)
    
    return dataset

# Replace this with your actual HDF file path
hdf_file_path = hdf4_file_path

# Convert the HDF file to an xarray Dataset
xarray_dataset = hdf_to_xarray(hdf_file_path)

# Now, xarray_dataset is an xarray Dataset containing all the subdatasets from the HDF file
xarray_dataset


In [54]:
from osgeo import gdal

# Open the HDF4 file
hdf4_dataset = gdal.Open(hdf4_file_path)

# List subdatasets
subdatasets = hdf4_dataset.GetSubDatasets()

# Open a specific subdataset
subdataset_index = 0  # Example: Change this to the index of the subdataset you want to open
subdataset_path = subdatasets[subdataset_index][0]
subdataset_ds = gdal.Open(subdataset_path)
subdatasets[subdataset_index][0]
# Now you can work with `subdataset_ds` directly using GDAL functions


'HDF4_EOS:EOS_SWATH:"C:/Users/isxzl/OneDrive/Code/AutoGeo/data/hdf4/MOD04_L2.A2023001.0350.061.2023001180312.hdf":mod04:Scan_Start_Time'

In [56]:
subdataset_ds.ReadAsArray().shape

(203, 135)